In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib
import seaborn as sns
import pynbody as pb

# g8.26e11

In [ ]:
s = pb.load('./data/g8.26e11/snap_2000/output/8.26e11.02000')
s.physical_units()
h = s.halos()

## Fe/H vs O/Fe

In [ ]:
#let's begin by centering it 
pb.analysis.angmom.faceon(h[1])

#create and apply the filter
disk_filt = pb.filt.LowPass('r', '100.00 kpc')
s_filt = s[disk_filt]

In [ ]:
feh = s_filt.s['feh']
ofe = s_filt.s['ofe']

In [ ]:
ofe.max()

In [ ]:
ofe_all = s.s['ofe']
ofe_all.max()

In [ ]:
plt.hist(ofe_all, bins='sqrt');

In [ ]:
sns.kdeplot(ofe_all)

# g7.55e11

In [2]:
s = pb.load('./data/g7.55e11/snap_1024/output/7.55e11.01024')
s.physical_units()
h = s.halos()
pb.analysis.angmom.faceon(h[1])

In [ ]:
ofe_all = s.s['ofe']

plt.hist(ofe_all, 'sqrt');
plt.xlabel('[O/Fe]')

In [ ]:
#create and apply the filter
# disk_filt = pb.filt.LowPass('r', '30.00 kpc')
disk_filt = pb.filt.SolarNeighborhood(r1=f'{0.00} kpc', r2=f'{3.00} kpc', height=f'{5} kpc', cen=(0, 0, 0)) 
s_filt = s[disk_filt]

feh_disk = s_filt.s['feh']
ofe_disk = s_filt.s['ofe']

plt.hist(ofe_disk, 'sqrt')
plt.xlabel('[O/Fe]')

In [ ]:
#let's take the 99.9 percentile 
percentile_feh = np.percentile(feh_disk, 0.01)
percentile_ofe = np.percentile(ofe_disk, 0.01)
print(percentile_feh)
print(percentile_ofe)

df = pd.DataFrame(data=(zip(feh_disk[feh_disk>percentile_feh], ofe_disk[ofe_disk>percentile_ofe])),
                  columns= ['Fe/H', 'O/Fe'])


In [ ]:
gs = gridspec.GridSpec(1, 2)

plt.figure(figsize=(10, 4))
ax = plt.subplot(gs[0]) 
plt.hist(df['Fe/H'], bins = 'sqrt', density=True)
plt.xlabel('[Fe/H]')
plt.grid(linestyle='dotted')

ax = plt.subplot(gs[1])
plt.hist(df['O/Fe'], density=True)
plt.xlabel('[O/Fe]')
plt.grid(linestyle='dotted')
plt.show()

plt.figure(figsize=(6, 3))
plt.hist2d(df['Fe/H'], df['O/Fe'], bins=(1000, 1000), norm=matplotlib.colors.LogNorm())
plt.colorbar(location='top', aspect=35)
plt.grid(linestyle='dotted')
plt.xlabel('[Fe/H]')
plt.ylabel('[O/Fe]')
plt.xlim(-0.6, 0.5)
plt.show()

In [ ]:
r_interval = [[0.001, 3.0]]
gs = gridspec.GridSpec(1, len(r_interval), wspace=0)
disk_filt = pb.filt.SolarNeighborhood(r1=f'{r_interval[0][0]} kpc', r2=f'{r_interval[0][1]} kpc', height=f'{5} kpc', cen=(0, 0, 0)) 
feh = s[disk_filt].s['feh']
ofe = s[disk_filt].s['ofe']

feh_array = np.array(feh)
ofe_array = np.array(ofe)
#let's take the 99.9 percentile 
percentile_feh = np.percentile(feh_array, n_percentile)
percentile_ofe = np.percentile(ofe_array, n_percentile)

#dataFrame for easy access
df = pd.DataFrame(data=( zip(feh_array[feh_array>percentile_feh], ofe_array[ofe_array>percentile_ofe]) ), 
                  columns= ['Fe/H', 'O/Fe'])
#plotting
ax = plt.subplot(gs[0])
plt.hist2d(df['Fe/H'], df['O/Fe'], bins=(500, 500), norm=matplotlib.colors.LogNorm())
plt.xlabel('[Fe/H]')
plt.ylabel('[O/Fe]')
plt.xlim(-0.8, 0.6)
plt.ylim(-0.16, 0.058)
x_ticks = ax.xaxis.get_major_ticks()
x_ticks[-1].label1.set_visible(False)
plt.colorbar(location='top', shrink=0.9)
plt.text(x = -0.2, y = 0.04, s=rf'{r1} < R < {r2} kpc')
